In [73]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.losses import SparseCategoricalCrossentropy
import cv2
import os
from tqdm import tqdm
import numpy as np
# import pandas as pd
import re

In [57]:
ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
suits = ['C', 'D', 'H', 'S']


cards = [f'{rank}{suit}' for suit in suits for rank in ranks]
one_hot_matrix = np.eye(len(cards), dtype=int)

one_hot_matrix.shape

(52, 52)

In [58]:
# pd.DataFrame(one_hot_matrix, index=cards, columns=cards)

In [59]:
def card_to_one_hot(card):
    if card in cards:
        index = cards.index(card)
        return one_hot_matrix[index]
    else:
        raise ValueError(f"Card '{card}' is not valid. Use format like 'AC' for Ace of Clubs.")

In [60]:
images_paths = list(os.listdir('./Card Images/Preprocessed Images'))

In [61]:
imgs = []
for i in tqdm(images_paths):
    if 'JOKER' in i:
        continue
    imgs.append(cv2.imread(f'./Card Images/Preprocessed Images/{i}'))

100%|██████████| 2757/2757 [00:27<00:00, 98.91it/s] 


In [62]:
x = np.array(imgs)

In [63]:
x.shape

(2706, 540, 960, 3)

In [64]:
cards_y = []
for i in images_paths:
    if 'JOKER' in i:
        continue
    match = re.match(r'^(A|J|K|Q|\d+)([CSHD])', i)
    face = match.group(1)
    suit = match.group(2)

    cards_y.append(f'{face}{suit}')

In [65]:
y = np.array([card_to_one_hot(i) for i in cards_y])

In [66]:
y.shape

(2706, 52)

In [67]:
# could try relu or swish

In [78]:
model = Sequential()
img_size = (540, 960, 3)
model.add(Conv2D(input_shape=img_size,filters=64,kernel_size=(3,3),padding='same', activation='swish'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(input_shape=(224, 244),filters=64,kernel_size=(3,3),padding='same', activation='swish')) # may need adjustment
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='swish'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dense(units=256))
model.add(Dense(units=52))

In [79]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_77 (Conv2D)          (None, 540, 960, 64)      1792      
                                                                 
 conv2d_78 (Conv2D)          (None, 540, 960, 64)      36928     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 270, 480, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_79 (Conv2D)          (None, 270, 480, 64)      36928     
                                                                 
 conv2d_80 (Conv2D)          (None, 270, 480, 64)      36928     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 135, 240, 64)     0         
 g2D)                                                 

In [80]:
model.compile(loss=SparseCategoricalCrossentropy())

In [81]:
model.fit(x, y, epochs=1)

: 